# LSTM으로 텍스트 생성하기

## 시퀀스 데이터를 어떻게 생성할까?

이전 토큰들이 주어졌을 때 다음 토큰의 확률을 모델링할 수 있는 네트워크를 언어 모델이라고 부릅니다. 언어 모델은 언어의 통계적 구조인 잠재공간을 탐색합니다. 초기 텍스트 문자열을 주입하고 새로운 글자나 단어를 생성합니다. 생성된 출력은 다시 입력 데이터로 추가됩니다. 이 과정을 반복합니다. 이 LSTM을 글자 수준의 신경망 언어 모델이라고 부릅니다.

## 샘플링 전략의 중요성

텍스트를 생성할 때 다음 글자를 선택하는 방법이 아주 중요합니다. 단순한 방법은 항상 가장 높은 확률을 가진 글자를 선택하는 탐욕적 샘플링입니다. 좀 더 흥미로운 방법을 사용하면 놀라운 선택이 만들어집니다. 다음 글자의 확률 분포에서 샘플링하는 과정에 무작위성을 주입하는 방법입니다. 이를 확률적 샘플링이라고 부릅니다.  

샘플링 과정에서 확률의 양을 조절하기 위해 소프트맥스 온도라는 파라미터를 사용합니다. 이 파라미터는 샘플링에 사용되는 확률 분포의 엔트로피를 나타냅니다.

In [1]:
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5): # 오리지널은 전체 합이 1인 넘파이 배열
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

## 글자 수준의 LSTM 텍스트 생성 모델 구현

In [2]:
# 데이터 전처리

import keras
import numpy as np

path = keras.utils.get_file(
'nietzsche.txt',
origin = 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('말뭉치 크기:', len(text))

Using TensorFlow backend.
C:\Users\Administrator\.conda\envs\multicampus\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Administrator\.conda\envs\multicampus\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Administrator\.conda\envs\multicampus\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Admin

606208/600901 [==============================] - 1s 2us/step
말뭉치 크기: 600893


In [3]:
maxlen = 60
step = 3

sentences = []

next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('시퀀스 개수:', len(sentences))

chars = sorted(list(set(text)))
print('고유한 글자:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)
print('벡터화...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

시퀀스 개수: 200278
고유한 글자: 58
벡터화...


In [4]:
# 네트워크 구성

from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [5]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

1. 지금까지 생성된 텍스트를 주입하여 모델에서 다음 글자에 대한 확률 분포를 뽑습니다.
2. 특정 온도로 이 확률 분포의 가중치를 조정합니다.
3. 가중치가 조정된 분포에서 무작위로 새로운 글자를 샘플링합니다.
4. 새로운 글자를 생성된 텍스트의 끝에 추가합니다.

In [6]:
# 언어 모델 훈련과 샘플링

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1, 60):
    print('에포크', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('---- 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

에포크 1
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/1
200278/200278 [==============================] - 260s 1ms/step - loss: 2.0064
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
---- 온도: 0.2
the slowly ascending ranks and classes, in which,
through for the stristing that the same and stander that in the soul that in the self so the stridity, and and and standed that should that in the strick and strict, and that in the stristion of the self--the same and strict and string of the constitical and the standed that in the struld of the condent that the supposition of the same and and the strict and desting that in the same and the bect, one all th
---- 온도: 0.5
the slowly ascending ranks and classes, in which,
through forments of the stroute of a strigated the surdicate of the same intermance of puris that prope and of the subdire the somethings than of the matter of the religion doward to the condence of 

KeyboardInterrupt: 

## 정리

- 이전의 토큰이 주어지면 다음 토큰(들)을 예측하는 모델을 훈련하여 시퀀스 데이터를 생성할 수 있습니다.
- 텍스트의 경우 이런 모델을 언어 모델이라고 부릅니다. 단어 또는 글자 단위 모두 가능합니다.
- 다음 토큰을 샘플링할 때 모델이 만든 출력에 집중하는 것과 무작위성을 주입하는 것 사이에 균형을 맞추어야 합니다.
- 이를 위해 소프트맥스 온도 개념을 사용합니다. 항상 다양한 온도를 실험해서 적절한 값을 찾습니다.

# 딥드림

딥드림은 합성곱 신경망이 학습한 표현을 사용하여 예술적으로 이미지를 조작하는 기법입니다.

- 딥드림에서는 특정 필터가 아니라 전체 층의 활성화를 최대화합니다. 한꺼번에 많은 특성을 섞어 시각화합니다.
- 빈 이미지나 노이즈가 조금 있는 입력이 아니라 이미 가지고 있는 이미지를 사용합니다. 그 결과 기존 시각 패턴을 바탕으로 이미지의 요소들을 다소 예술적인 스타일로 왜곡시킵니다.
- 입력 이미지는 시각 품질을 높이기 위해 여러 다른 스케일로 처리합니다.

## 케라스 딥드림 구현

In [8]:
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet',
                                include_top=False)




87916544/87910968 [==============================] - 64s 1us/step


In [9]:
layer_contributions = {
    'mixed2' : 0.2,
    'mixed3' : 3.,
    'mixed4' : 2.,
    'mixed5' : 1.5,
}

In [10]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [11]:
dream = model.input

grads = K.gradients(loss, dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...', i, '번째 손실:', loss_value)
        x += step * grad_values
    return x

In [19]:
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
              float(size[0]) / img.shape[1],
              float(size[1]) / img.shape[2],
              1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    image.save_img(fname, pil_img)
    
def preprocessing_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
        
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [20]:
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.

base_image_path = './datasets/original_photo_deep_dream.jpg'

img = preprocessing_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('처리할 이미지 크기', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='./datasets/final_dream.png')

처리할 이미지 크기 (178, 178)
... 0 번째 손실: 0.65961766
... 1 번째 손실: 1.0167831
... 2 번째 손실: 1.4680852
... 3 번째 손실: 2.004314
... 4 번째 손실: 2.6085532
... 5 번째 손실: 3.094368
... 6 번째 손실: 3.5717893
... 7 번째 손실: 4.0804224
... 8 번째 손실: 4.477044
... 9 번째 손실: 4.8836703
... 10 번째 손실: 5.3870115
... 11 번째 손실: 5.730486
... 12 번째 손실: 6.0587974
... 13 번째 손실: 6.525957
... 14 번째 손실: 6.8514767
... 15 번째 손실: 7.2965965
... 16 번째 손실: 7.7134266
... 17 번째 손실: 8.051285
... 18 번째 손실: 8.430019
... 19 번째 손실: 8.819474
처리할 이미지 크기 (250, 250)
... 0 번째 손실: 2.229415
... 1 번째 손실: 3.5246768
... 2 번째 손실: 4.5975065
... 3 번째 손실: 5.439809
... 4 번째 손실: 6.2940874
... 5 번째 손실: 7.0495
... 6 번째 손실: 7.7875357
... 7 번째 손실: 8.472773
... 8 번째 손실: 9.100412
... 9 번째 손실: 9.730979
처리할 이미지 크기 (350, 350)
... 0 번째 손실: 2.3907166
... 1 번째 손실: 3.5997496
... 2 번째 손실: 4.7493773
... 3 번째 손실: 5.8188076
... 4 번째 손실: 6.944591
... 5 번째 손실: 8.298418
... 6 번째 손실: 9.89694


## 정리

- 딥드림은 네트워크가 학습한 표현을 기반으로 컨브넷을 거꾸로 실행하여 입력 이미지를 생성합니다.
- 재미있는 결과가 만들어지고, 때로는 환각제 떄문에 시야가 몽롱해진 사람이 만든 이미지 같기도 합니다.
- 이 과정은 이미지 모델이나 컨브넷에 국한되지 않습니다. 음성, 음악 등에도 적용될 수 있습니다.

# 뉴럴 스타일 트랜스퍼

참조 이미지의 스타일을 적용하면서 원본 이미지의 콘텐츠를 보존하는 것이다.

## 콘텐츠 손실

네트워크에 있는 하위 층의 활성화는 이미지에 관한 국부적인 정보를 담고 있습니다. 반면에 상위 층의 활성화일수록 점점 전역적이고 추상적인 정보를 담게 됩니다. 다른 방식으로 생각하면 컨브넷 층의 활성화는 이미지를 다른 크기의 콘텐츠로 분해한다고 볼 수 있습니다. L2 노름이 콘텐츠 손실로 사용하기에 좋습니다.

## 스타일 손실

콘텐츠 손실은 하나의 상위 층만 사용합니다. 스타일 손실은 컨브넷의 여러 층을 사용합니다. 하나의 스타일이 아니라 참조 이미지에서 컨브넷이 추출한 모든 크기의 스타일을 잡아야 합니다. 활성화 출력의 그람 행렬을 스타일 손실로 사용했습니다. 그람 행렬은 층의 특성 맵들의 내적입니다.

- 콘텐츠를 보존하기 위해 타깃 콘텐츠 이미지와 생성된 이미지 사이에서 상위 층의 활성화를 비슷하게 유지합니다. 이 컨브넷은 타킷 이미지와 생성된 이미지에서 동일한 것을 보아야 합니다.
- 스타일을 보존하기 위해 저수준 층과 고수준 층에서 활성화 안에 상관관계를 비슷하게 유지합니다. 특성의 상관관계는 텍스처를 잡아냅니다. 생성된 이미지와 스타일 참조 이미지는 여러 크기의 텍스처를 공유할 것입니다.

## 케라스에서 뉴럴 스타일 트랜스퍼 구현하기

1. 스타일 참조 이미지, 타깃 이미지, 생성된 이미지를 위해 VGG19의 층 활성화를 동시에 계산하는 네트워크를 설정합니다.
2. 세 이미지에서 계산할 층 활성화를 사용하여 앞서 설명한 손실 함수를 정의합니다. 이 손실을 최소화하여 스타일 트랜스퍼를 구현할 것입니다.
3. 손실 함수를 최소화할 경사 하강법 과정을 설정합니다.

In [22]:
from keras.preprocessing.image import load_img, img_to_array, save_img

target_image_path = './datasets/portrait.png'
style_reference_image_path = './datasets/popova.jpg'

width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

In [23]:
import numpy as np
from keras.applications import vgg19

def preprocessing_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [25]:
from keras import backend as K

target_image = K.constant(preprocessing_image(target_image_path))
style_reference_image = K.constant(preprocessing_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_image,
                              style_reference_image,
                              combination_image], axis=0)

model = vgg19.VGG19(input_tensor=input_tensor,
                   weights='imagenet',
                   include_top=False)
print('모델 로드 완료')

80142336/80134624 [==============================] - 29s 0us/step
모델 로드 완료


In [27]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [26]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2,0,1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C) / (4. * (channels ** 2) * (size ** 2)))

In [28]:
def total_variation_loss(x):
    a = K.square(x[:, :img_height - 1, :img_width -1, :] - x[:, 1:, :img_width -1, :])
    b = K.square(x[:, :img_height -1, :img_width -1, :] - x[:, :img_height -1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [31]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = ['block1_conv1',
               'block2_conv1',
               'block3_conv1',
               'block4_conv1',
               'block5_conv1']
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features, combination_features)

for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
    
loss += total_variation_weight * total_variation_loss(combination_image)

In [32]:
grads = K.gradients(loss, combination_image)[0]

fetch_loss_and_grads = K.function([combination_image], [loss, grads])

class Evaluator(object):
    
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
        
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values
    
evaluator = Evaluator()

In [33]:
from scipy.optimize import fmin_l_bfgs_b
import time

result_prefix = 'style_transfer_result'
iterations = 20

x = preprocessing_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('반복 횟수:', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss,
                                    x,
                                    fprime=evaluator.grads,
                                    maxfun=20)
    print('현재 손실 값:', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    save_img(fname, img)
    print('저장 이미지:', fname)
    end_time = time.time()
    print('%d 번째 반복 완료: %ds' % (i, end_time - start_time))

반복 횟수: 0


KeyboardInterrupt: 

## 정리
- 스타일 트랜스퍼는 참조 이미지의 스타일을 적용하면서 타깃 이미지의 콘텐츠를 보존하여 새로운 이미지를 만드는 방법입니다.
- 콘텐츠는 컨브넷 상위 층의 활성화에서 얻을 수 있습니다.
- 스타일은 여러 컨브넷 층의 활성화 안에 내재된 상관관계에서 얻을 수 있습니다.
- 딥러닝에서는 사전 훈련된 컨브넷으로 손실을 정의하고 이 손실을 최적화하는 과정으로 스타일 트랜스퍼를 구성할 수 있습니다.
- 이런 기본 아이디어에서 출발하여 다양한 변종과 개선이 가능합니다.

# 변이형 오토인코더를 사용한 이미지 생성

## 이미지의 잠재 공간에서 샘플링하기

이미지 생성의 핵심 아이디어는 각 포인트가 실제와 같은 이미지로 매핑될 수 있는 저차원 잠재공간의 표현을 만드는 것입니다. 잠재 공간의 한 포인트를 입력으로 받아 이미지를 출력하는 모듈을 생성자 또는 디코더라고 부릅니다.  
VAE는 구조적인 잠재 공간을 학습하는 데 뛰어납니다. 이 공간에서 특정 방향은 데이터에서 의미 있는 변화의 방향을 인코딩합니다.  
GAN은 매우 실제 같은 이미지를 만듭니다. 여기에서 만든 잠재 공간은 구조적이거나 연속성이 없을 수 있습니다.

## 이미지 변형을 위한 개념 벡터

잠재 공간이나 임베딩 공간이 주어지면 이 공간의 어떤 방향은 원본 데이터의 흥미로운 변화를 인코딩한 축일 수 있습니다.

## 변이형 오토인코더

변이형 오토인코더는 딥러닝과 베이즈 추론의 아이디어를 혼합한 오토인코더의 최신 버전입니다. 다시 말해 오토인코더는 원본 입력을 재구성하는 방법을 학습합니다. VAE는 이미지를 어떤 통계 분포의 파라미터로 변환합니다.

1. 인코더 모듈이 입력 샘플 input_img를 잠재 공간의 두 파라미터 z_mean과 z_log_var로 변환합니다.
2. 입력 이미지가 생성되었다고 가정한 잠재 공간의 정규 분포에서 포인트 z를 z = z_mean + exp(0.5 * z_log_var) * epsilon처럼 무작위로 샘플링합니다. epsilon은 작은 값을 가진 랜덤 텐서입니다.
3. 디코더 모듈은 잠재 공간의 이 포인트를 원본 입력 이미지로 매핑하여 복원합니다.

VAE의 파라미터는 2개의 손실 함수로 훈련합니다. 재구성 손실과 규제 손실입니다.

In [34]:
import keras
from keras import layers
from keras import backend as K
from keras.models import Model
import numpy as np

img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2

input_img = keras.Input(shape=img_shape)

x = layers.Conv2D(32, 3, padding='same', activation='relu')(input_img)
x = layers.Conv2D(64, 3, padding='same', activation='relu', strides=(2,2))(x)
x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
shape_before_flattening = K.int_shape(x)

x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)

z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

In [35]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

In [36]:
decoder_input = layers.Input(K.int_shape(z)[1:])

x = layers.Dense(np.prod(shape_before_flattening[1:]),
                activation='relu')(decoder_input)
x = layers.Reshape(shape_before_flattening[1:])(x)
x = layers.Conv2DTranspose(32, 3,
                          padding='same',
                          activation='relu',
                          strides=(2, 2))(x)
x = layers.Conv2D(1, 3,
                 padding='same',
                 activation='sigmoid')(x)

decoder = Model(decoder_input, x)

z_decoded = decoder(z)

In [38]:
class CustomVariationlLayer(keras.layers.Layer):
    
    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=1)
        return K.mean(xent_loss + kl_loss)
    
    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return x
    
y = CustomVariationlLayer()([input_img, z_decoded])

In [39]:
from keras.datasets import mnist

vae = Model(input_img, y)
vae.compile(optimizer='rmsprop', loss=None)
vae.summary()

(x_train, _), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(x_test.shape + (1,))

vae.fit(x=x_train, y=None,
       shuffle=True,
       epochs=10,
       batch_size=batch_size,
       validation_data=(x_test, None))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 28, 28, 32)   320         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_96 (Conv2D)              (None, 14, 14, 64)   18496       conv2d_95[0][0]                  
__________________________________________________________________________________________________
conv2d_97 (Conv2D)              (None, 14, 14, 32)   18464       conv2d_96[0][0]                  
__________________________________________________________________________________________________
conv2d_98 

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm

n = 15
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        z_sample = np.tile(z_sample, batch_size).reshape(batch_size, 2)
        x_decoded = decoder.predict(z_sample, batch_size=batch_size)
        
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i+1) * digit_size, j * digit_size: (j+1) * digit_size] = digit
        
plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.show()

## 정리

- 딥러닝으로 이미지 데이터셋에 대한 통계 정보를 담은 잠재 공간을 학습하여 이미지를 생성할 수 있습니다. 잠재 공간에서 포인트를 샘플링하고 디코딩하면 이전에 본 적 없는 이미지를 생성합니다. 이를 수행하는 주요 방법은 VAE와 GAN입니다.
- VAE는 매우 구조적이고 연속적인 잠재 공간의 표현을 만듭니다. 이런 이유로 잠재 공간 안에서 일어나는 모든 종류의 이미지 변형 작업에 잘 맞습니다. 다른 얼굴로 바꾸기, 찌푸린 얼굴을 웃는 얼굴로 변형하기 등입니다. 잠재공간을 가로질러 이미지가 변환하는 잠재 공간 기반의 애니메이션에도 잘 맞습니다. 시작 이미지가 연속적으로 다른 이미지로 부드럽게 바뀌는 것을 볼 수 있습니다.
- GAN은 실제 같은 단일 이미지를 생성할 수 있지만 구조적이고 연속적인 잠재 공간을 만들지 못합니다.

# 적대적 생성 신경망 소개

- 생성자 네트워크 : 랜덤 벡터(잠재 공간의 무작위한 포인트)를 입력으로 받아 이를 합성된 이미지로 디코딩합니다.
- 판별자 네트워크 : 이미지(실제 또는 합성 이미지)를 입력으로 받아 훈련 세트에서 온 이미지인지, 생성자 네트워크가 만든 이미지인지 판별합니다.

## GAN 구현 방법

1. generator 네트워크는 (latent_dim,)크기의 벡터를 (32,32,3) 크기의 이미지로 매핑합니다.
2. discriminator 네트워크는 (32, 32, 3) 크기의 이미지가 진짜일 확률을 추정하여 이진 값으로 매핑합니다.
3. 생성자와 판별자를 연결하는 gan 네트워크를 만듭니다. gan(x) = discriminator(generator(x))입니다. 이 gan 네트워크는 잠재 공간의 벡터를 판별자의 평가로 매핑합니다. 즉 판별자는 생성자에게 잠재 공간의 벡터를 디코딩한 것이 얼마나 현실적인지 평가합니다.
4. "진짜"/"가짜" 레이블과 함께 진짜 이미지와 가짜 이미지 샘플을 사용하여 판별자를 훈련합니다. 일반적인 이미지 분류 모델을 훈련하는 것과 동일합니다.
5. 생성자를 훈련하려면 gan 모델의 손실에 대한 생성자 가중치의 그래디언트를 사용합니다. 이 말은 매 단계마다 생성자에 의해 디코딩된 이미지를 판별자가 "진짜"로 분류하도록 만드는 방향으로 생성자의 가중치를 이동한다는 뜻입니다. 다른 말로 하면 판별자를 속이도록 생성자를 훈련합니다.

## 훈련 방법

- 생성자의 마지막 활성화로 다른 종류의 모델에서 널리 사용하는 sigmoid 대신 tanh 함수를 사용합니다.
- 균등 분포가 아니고 정규 분포를 사용하여 잠재 공간에서 포인트를 샘플링합니다.
- 무작위성은 모델을 견고하게 만듭니다. GAN 훈련은 동적 평형을 만들기 때문에 여러 방식으로 갇힐 가능성이 높습니다. 훈련하는 동안 무작위성을 주입하면 이를 방지하는 데 도움이 됩니다. 무작위성은 두 가지 방법으로 주입합니다. 판별자에 드롭아웃을 사용하거나 판별자를 위해 레이블에 랜덤 노이즈를 추가합니다.
- 희소한 그래디언트 GAN 훈련을 방해할 수 있습니다. 딥러닝에서 희소는 종종 바람직한 현상이지만 GAN에서는 그렇지 않습니다. 최대 풀링 대신 스트라이드 합성곱을 사용하여 다운샘플링을 하는 것이 좋습니다. 또 ReLU 활성화 대신 LeakyReLU 층을 사용하세요. ReLU와 비슷하지만 음수의 활성화 값을 조금 허용하기 때문에 희소가 조금 완화됩니다.
- 생성자에서 픽셀 공간을 균일하게 다루지 못하여 생성된 이미지에서 체스판 모양이 종종 나타납니다. 이를 해결하기 위해 생성자나 판별자에서 스트라이드 Conv2DTranspose나 Conv2D를 사용할 때 스트라이드 크기로 나누어질 수 있는 커널 크기를 사용합니다.

## 생성자

In [41]:
import keras
from keras import layers
import numpy as np

latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape=(latent_dim,))
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_101 (Conv2D)          (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

## 판별자

In [42]:
discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)

x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

discriminator_optimizer = keras.optimizers.RMSprop(
lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_105 (Conv2D)          (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_106 (Conv2D)          (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_107 (Conv2D)          (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 6, 6, 128)         0         
__________

## 적대적 네트워크

In [43]:
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

## DCGAN 훈련 방법

1. 잠재 공간에서 무작위로 포인트를 뽑습니다.
2. 이 랜덤 노이즈를 사용하여 generator에서 이미지를 생성합니다.
3. 생성된 이미지와 진짜 이미지를 섞습니다.
4. 진짜와 가짜가 섞인 이미지와 이에 대응하는 타깃을 사용하여 discriminator를 훈련합니다. 타깃은 "진짜(실제 이미지일 경우)" 또는 "가짜(생성된 이미지일 경우)" 입니다.
5. 잠재 공간에서 무작위로 새로운 포인트를 뽑습니다.
6. 이 랜덤 벡터를 사용하여 gan을 훈련합니다. 모든 타깃은 "진짜"로 설정합니다. 판별자가 생성된 이미지를 모두 "진짜 이미지"라고 예측하도록 생성자의 가중치를 업데이트합니다. 결국 생성자는 판별자를 속이도록 훈련합니다.

In [ ]:
import os
from keras.preprocessing import image

(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

x_train = x_train[y_train.flatten() == 6]

x_train = x_train.reshape((x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20
save_dir = './datasets/gan_images/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
    
start = 0
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    generated_images = generator.predict(random_latent_vectors)
    
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)
    d_loss = discriminator.train_on_batch(combined_images, labels)
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    misleading_targets = np.zeros((batch_size, 1))
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
    if step % 100 == 0:
        gan.save_weights('gan.h5')
        
        print('판별자 손실:', d_loss)
        print('생성자 손실:', a_loss)
        
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))
        
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'real_frog' + str(step) + '.png'))

## 정리

- GAN은 생성자 네트워크와 판별자 네트워크가 연결되어 구성됩니다. 판별자는 생성자의 출력과 훈련 데이터셋에서 가져온 진짜 이미지를 구분하도록 훈련됩니다. 생성자는 판별자를 속이도록 훈련됩니다. 놀랍게도 생성자는 훈련 세트의 이미지를 직접 보지 않습니다. 데이터에 관한 정보는 판별자에서 얻습니다.
- GAN은 훈련하기 어렵습니다. GAN 훈련이 고정된 손실 공간에서 수행하는 단순한 경사하강법 과정이 아니라 동적 과정이기 때문입니다. GAN을 올바르게 훈련하려면 경험적으로 찾은 여러 기교를 사용하고 많은 튜닝을 해야 합니다.
- GAN은 매우 실제 같은 이미지를 만들 수 있습니다. VAE와 달리 학습된 잠재 공간이 깔끔하게 연속된 구조를 가지지 않습니다. 잠재 공간의 개념 벡터를 사용하여 이미지를 변형하는 등 실용적인 특정 애플리케이션에는 잘 맞지 않습니다.

# 요약

- 한 번에 하나의 타임스텝씩 시퀀스 데이터를 생성하는 방법, 텍스트 생성이나 음표 하나씩 음악을 생성하거나 어떤 시계열 데이터를 생성하는 곳에 적용할 수 있습니다.
- 딥드림의 작동 원리. 컨브넷 층 활성화를 최대화하기 위해 입력 공간에 경사 상승법을 적용합니다.
- 스타일 트랜스퍼 적용 방법. 콘텐츠 이미지와 스타일 이미지가 연결되어 재미있는 결과를 만듭니다.
- GAN과 VAE가 무엇인지와 이를 사용하여 새로운 이미지를 만드는 방법. 잠재 공간의 개념 벡터를 사용하여 이미지를 변형하는 방법